# 20 Years Of Games Analysis- Exploratory Data Analysis

### -Ashwin

The dataset contains the data about the major games and releases on various platforms, with the month and year of release too. Score for each game is also provided. The data for games from 1996 to 2017 is present in the dataset. Some of the questions that I will try too answer in the Notebook are as follows:

 1. Major Platforms(Platforms with highest Games released)
 2. Major Genres of Games
 3. Masterpiece Analysis(Games with Full Ratings)
 4. Analysis Of the Year with the maximum releases.  etc

Hope you all like this Notebook. **Please Upvote** if this notebook helps you!!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import base64
import io
import codecs
from scipy.misc import imread
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))



# Any results you write to the current directory are saved as output.

In [ ]:
games=pd.read_csv('../input/ign.csv')

In [ ]:
games.head()

In [ ]:
games.isnull().sum()

In [ ]:
games.drop(['Unnamed: 0','url'],axis=1,inplace=True) #dropping the unneeded values
games.drop(games.index[516],inplace=True) #dropping the game released on 1970. It looked to be an outlier

In [ ]:
games.head(2)

### Let's check the Dataset

In [ ]:
games.isnull().sum()

Only the genre column has null values. Let's do some Analysis Now.

In [ ]:
games.dtypes

### Top Genre's

In [ ]:
games['genre'].value_counts()[:10].plot(kind='pie',autopct='%1.1f%%',shadow=True,explode=[0.1,0,0,0,0,0,0,0,0,0])
plt.title('Distribution Of Top Genre"s')
fig=plt.gcf()
fig.set_size_inches(7,7)
plt.show()

### Games Released on WeekDays

In [ ]:
games.groupby('release_day')['genre'].count().plot(color='y')
fig=plt.gcf()
fig.set_size_inches(12,6)

Seems that most of the games are released on 13-14 day of the month followed by the 17-18 of the month

### Let's Check the Highest Release in Months and Years

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(22,10))
sns.countplot(games['release_month'],ax=ax[0],palette='Set1').set_title('Releases On Months')
plt.ylabel('')
sns.countplot(games['release_year'],ax=ax[1],palette='Set1').set_title('Releases on Years')
plt.xticks(rotation=90)
plt.show()

**Observations:**

 1. The number of releases have always increased from August to November.
 2. Similarly the releases were constantly increasing from 2001 to 2008. After that there has been a decline in game releases.
 3. The year 2008 looks to be the best year for the games market. We will look into games of that year later.

### Score Distribution

In [ ]:
games['score'].hist(edgecolor='black')
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.axvline(games['score'].mean(),color='b',linestyle='dashed')

We can see that the majority of the scores are between 7-8. The mean score is around 6.8-6.9

### Top Platforms

In [ ]:
games['platform'].value_counts()[:10].plot.pie(autopct='%1.1f%%',shadow=True,explode=[0.1,0,0,0,0,0,0,0,0,0])
fig=plt.gcf()
fig.set_size_inches(7,7)
plt.title('Top Platforms For Games')

Clearly the PC games market is quite ahead of the others

### Average Scores Of Top Genre's By Years

In [ ]:
plt.subplots(figsize=(15,15))
max_genres=games.groupby('genre')['genre'].count()
max_genres=max_genres[max_genres.values>200]
max_genres.sort_values(ascending=True,inplace=True)
mean_games=games[games['genre'].isin(max_genres.index)]
abc=mean_games.groupby(['release_year','genre'])['score'].mean().reset_index()
abc=abc.pivot('release_year','genre','score')
sns.heatmap(abc,annot=True,cmap='RdYlGn',linewidths=0.4)
plt.title('Average Score By Genre"s')
plt.show()

Looking at the heatmap we can say that **Action-Rpg** games have a good ratings over the years as it has a single instance with mean score <7.

### Total Hits and Flops By Top Platforms

In [ ]:
clone=games.copy()
clone['score_phrase']=clone['score_phrase'].map({'Awful':'Flop','Bad':'Flop','Disaster':'Flop','Unbearable':'Flop','Painful':'Flop','Mediocre':'Good','Okay':'Good','Great':'Hit','Amazing':'Hit','Masterpiece':'Masterpiece','Good':'Good'})
clone=clone[['score_phrase','platform','genre','score']]
max_platforms=clone['platform'].value_counts().index[:10]
plat=clone[clone['platform'].isin(max_platforms)]
plat=plat.groupby(['platform','score_phrase'])['score'].count().reset_index()
plat=plat.pivot('platform','score_phrase','score')
plat.plot.barh(width=0.9)
fig=plt.gcf()
fig.set_size_inches(12,14)
plt.show()

**Observations:**

 - Not many masterpieces produced by the Top Platforms
 - The maximum masterpieces given by a console in Top Platforms is by PlayStation3
 - As PC games are released on a large scale, thus we have the maximum flops and hits from the same.

 

### Releases By Top Genres By Years

In [ ]:
new_genres=max_genres.sort_values(ascending=False)[:10]
top_genres=games[games['genre'].isin(new_genres[:10].index)]
top_genres=top_genres.groupby(['release_year','genre'])['score'].count().reset_index()
top_genres=top_genres.pivot('release_year','genre','score')
sns.heatmap(top_genres,annot=True,fmt='2.0f',cmap='RdYlGn',linewidths=0.4)
fig=plt.gcf()
fig.set_size_inches(11,11)
plt.title('Releases By Top Genres By Years')
plt.show()

Looking at the heatmap it is evident that **Action** games production has seen a substantial increase in the years 2005-2009 after which it has declined. Production of other genres is somewhat uneven in the years.

### Platforms With Highest Releases By Year

In [ ]:
new=games.groupby(['release_year','platform'])['score'].count().reset_index()
#new.columns=[['release_year','platform','count']]
new=new.sort_values(by='score',ascending=False)
new=new.drop_duplicates(subset=['release_year'],keep='first')
new=new.sort_values(by='release_year')
new.set_index('release_year',inplace=True)
new.plot.barh(color='#0154ff',width=0.8)
fig=plt.gcf()
fig.set_size_inches(10,10)
for i, p in enumerate(zip(new.platform, new['score'])):
    plt.text(s=p,x=1,y=i,fontweight='bold',color='white')
plt.show()

### Top Platforms With Their Favourite genres

In [ ]:
plat_genre=games.copy()
plat_genre=plat_genre[plat_genre['genre'].isin(new_genres.index)]
plat_genre=plat_genre.groupby(['platform','genre'])['score'].count().reset_index()
plat_genre=plat_genre[plat_genre['score']>10]
plat_genre=plat_genre.pivot('platform','genre','score')
plat_genre=plat_genre.dropna(thresh=8)
sns.heatmap(plat_genre,annot=True,fmt='2.0f',cmap='RdBu_r',linewidths='0.1')
fig=plt.gcf()
fig.set_size_inches(9,9)
plt.title('Platforms and Favorite Genres')
plt.show()

**Observations:**

 1. GameBoy Advance produces Action based games on a large scale as compared to other genres.
 2. PC leads in genres like Action, Adventure,Shooter and Strategy as compared to other platforms. Strategy based games are produced on a very large scale.
 3. Almost every Top-platform is having 100 action games in their arsenal.

**As we had seen earlier that the Year 2008 had the maximum releases. So lets do some analysis for Year 2008**

### Year 2008 In-Depth Analysis

In [ ]:
year08=games.copy()
year08=year08[year08['release_year']==2008]
genres_2008=year08['genre'].value_counts()[:15]

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))
year08['platform'].value_counts()[:10].plot.pie(autopct='%1.1f%%',ax=ax[0],shadow=True,explode=[0.1,0,0,0,0,0,0,0,0,0])
ax[0].set_title('2008 Top Platforms')
ax[0].set_ylabel('')
fig = plt.gcf()
year08['genre'].value_counts()[:10].plot.pie(autopct='%1.1f%%',ax=ax[1],shadow=True,explode=[0.1,0,0,0,0,0,0,0,0,0])
ax[1].set_title('2008 Top Genre')
ax[1].set_ylabel('')
plt.show()

**Observations:**

 1. The Action Genre has the maximum releases in Year 2008
 2. However **Wii** has maximum releases in Platforms as opposed to **PC** which has the overall maximum releases. 

### Score Distribution

In [ ]:
year08['score'].hist(color='#65ff65',edgecolor='black')
x=range(0,11,1)
plt.xticks(x)
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.axvline(year08['score'].mean(),linestyle='dashed',color='blue')

The mean score for Year 2008 is less than the mean score of the overall dataset. 

### Releases By Top Genre's

In [ ]:
plt.subplots(figsize=(18,6))
swarm=year08[year08['genre'].isin(genres_2008.index)]
sns.swarmplot(x='genre',y='score',data=swarm,cmap='Set 3')
plt.axhline(year08['score'].mean(),linestyle='dashed',color='blue')
plt.rcParams["axes.labelweight"] = "bold"
plt.show()

**Observations:**

 1. Action games again are released in highest amount in 2008.
 2. Also out of the Top Genres for 2008, **4 masterpieces** are from Action and 2 from Action-Adventure.
 3. **Action-Rpg** and **Action-Adventure** Games seems to be performing well as very few of them are below the average line.

### Top Platforms with Top Genre's Releases

In [ ]:
plt.subplots(figsize=(12,6))
dummy=year08[year08['genre'].isin(genres_2008.index)]
dummy=dummy.groupby(['platform','genre'])['score'].count().reset_index()
dummy=dummy.pivot('platform','genre','score')
dummy=dummy.dropna(thresh=10)
sns.heatmap(dummy,annot=True,linewidths=0.2)

Wii has produced the highest number of games for Year 2008 with the maximum being Action games , followed by Xbox-360.

Interesting to see that **Playstation** had kept itself away from **Simulation** games in 2008.

In [ ]:
xbox=b'...'
wii=b'iVBORw0KGgoAAAANSUhEUgAAAOEAAADhCAMAAAAJbSJIAAAAflBMVEX///8BAQEAAADV1dXz8/MdHR38/Pz5+fnq6urZ2dmYmJjg4OC6urrm5uarq6vOzs6IiIg2NjaxsbFsbGx1dXVVVVXBwcFMTEwnJydBQUFaWlrQ0NCCgoJlZWVGRkZ7e3sODg4zMzOOjo4bGxskJCSkpKQNDQ0tLS2cnJwWFhZHgIkuAAAJo0lEQVR4nO1daVvyOhB9CaLsIigoCKKC6P//gxdKW5plkplkUpab85GH9mSaZDJbkn//EhISEhISEhISEhISEhISEhISEhISEhISEhISEvzQ2S43rQN2y1F/UBPpoD9a7jLWzWTbicnUG/0JCZtRLyZfTtqSSb9Hsb7s+HP/+oaE/Q9P00h0R/R3RtJ+BKrfhkaV063iDdbeG0S63jJTDY1MBV+ML3rA2EYqxoxMvSeYKmPjJDvBImDGyjd4ulamjGzBxVVB0037y0LUtndgztVm4ZJ4MbSru3CiqZvowNUKZ1KwQfGKx1Ae5wgtqF44pKrgBckrApXqBCngnuqdR7Ic72jeMGLkhzwyDbmk+5etTnjiiT/PHC8gr0J1qlEmEQk9eGTiUqgINSozP/vxPNNo9kQbJglRalRi/vKh6VJp2BQqcew0PK0q3DqoMnEoVLQalUQku43UqVAQhStUghqtMv88EHmefGg4/Iy+l4D0GfLlR3MQMcwj9pocR2LS8CGtRypTiMGPNRKNxJTFaufNc2Ca+Rr87ZW/gHviGZ7pN4Ank7FLnfYHPHyFyEfSAXdhTFmI4YMahOs8WwIlWFos2SSQ6Sjj5n3cabfv3Gi3m+OPv2D5GnjTpsfA1ciEpICHEqcBSB6Fi7I26Y58KJuKqQvPAlwnMszC8wEzE/0M0kuBEPdOCb3ttYggTFiE7XaBXSjE87Q36AzfEG0TO5eAfcRL4uhBmK+0AhFmudtRdPnXB6FeRtv+tPPYH3YnrfhCVleAu1e3iB92AR0G216cUVOay4PhU1wZxbxK59aDLtNtbM/2vJrCIZ1lRBnVFW7o7kS7/W0bpLCP2cPoAD9oVoq7E+fGNuZ4sKUklxaXyC+4goCWeXEaJELYXDdYk7qCBAO3DvCC5ri7g5zWOAr4gdwh+/u3KCJqNgpiItqC/NBYEwLh0NIDuQj49KFFmz6CAjbdAnpHIO3NVYfcEmGSwFnTD/PT2ADIw5pfRDXpco+x3GAv8RuQ0GEmlOjwa1Q1RojJpsApd8D3JeToPbI5zuY+kb+hEFAZytYsIKUYoBVBxKWzD7RHoNy+WRmSEgK+WQdre3fFJ0aXTQBmjXkSI/VoAXJyE9Ng8drtT8f4gCq0XpjXCrGiCGg33fEiycLovqjDNQUm1sgsIS27arFsCfJ9rt7MpZDlP3arN0sQGQhIGRdsbJC1RGiobt/sbUa5mMMWVjdbQjpgtY94MzXN7PwSB2nwMBWVJf4RWL3mpa3aAWwMc79Mza/rEiVsB0pYtUcGxvLZyuoBxiSM/oXZZKNXxgVNRHl9N4WexLf0DyCXazTczIs1aB6AmAVlVxUnbaVLqNjIZkvcFFS8AwwaqoD0QqMqnVpzpM1qLagNrHEGRx+YhnQJfcphSjrVxtdmtW5hAcNUn13maL4y6jEIsb41vabZWfoEAyTUNaQ5CHEFEjbMDdfKFgBbpO5RqrZfH6WaUQ01XA3wNKGlkyxhkKZ5VV7mq2n2f1QjS1AI60pXC4M9DX358674BndTrKV/gNVbmlr+g/5Yt9VWbZi/1aYPePCflFKqDIgEpFXCSjwXsrxfTpb3j8XDktf8BRzsJhZwMXhP44yyB/tGuX81+LDMCDVVCsfKz+IBr+azV7sH/DZ/0bdD2hoOq/izRTHUmIXhF/tL5LUVTqsQK6hjpKDIkajjU5IGscXK7RlHBd4lvramkqOJx8ckY6VnTf0S9sJECCZWvQlS2kDKB5hdp+Kf543qf1YJFgQRJWPF3jL0LooomRnZvjSHPM2PVmP79kUMW39/DxlGAVATuoT6Zck8cmhApD7VzeRwaB8XX/0q2bjuPcWISn9EdpYOjyz3qdGnp4wWrvJvZy9GSeM3NP8Ub1JUuwXyIqW/2zPd7c8oAup9SNgpUekVdwWHHG7XESNveKRVF2NCCKFimaJU8F4GqBsHL/GqvkYKF2VBPPm2SP0kxKdJxkMKKJaAmvdGsewrKw26EGYvyvujXH25dTgxoZAnB2k7T8X2pvT8oYL2a/jY6fUW/d/Jdw0VtJU4ygOpKEn8lM/R2lh3EbSYFAO1aY7+wk+WE8mnnfVVvB8KkxeDQa8PJoXBJ4tQqNWzuAT4jpjSu8Ash1eJ0kq5xE0kLCiVFFCSeP0ozYWQTMNFowyPRyrRPj9KDzFC4etloIgwBW9svlgUycbr3nFoQ1EaNbhVAUsvH+HhXytyLz8w43fJyM02lnTRZSI328wF7DeBPFJzs2ZpGdi/WbO0UVRDX/URA3bkqYso0fjLQB5tixOOvwjkzkWcrZEXgTxDf7POUxkxvVnnqXQQb9a1KIvbblnC9c1LKP4vEjK+kAl8DToIeM/0vkPLZqNtf9EMwaK/Hc3YhDxKSMw8wS9bTn3OEjThYUpOwkCNOrzOvB2I/KoZ710eHZ7jUtgk5DrDv4ovDhG5JAw9ltUM++UPyJbds0jIcHy/EQwlLFmdQ7CmiXWBB0cEKZMweLXQSl74EBx+YFnxlW0uvADOIqFKGPSWGNeTnBBYk8til0ZYJ6oI2evHIyHbbQEQwlq3zt4RdrR1nIXihKCcQ14GHxJrY79eRkdI4WoeiQpQyagzwAIRUs+Ux0sJBf7aG5aO5nEg4AixPKrvXxIFH8jECcouErWBx4MC/YeB7/0uRPjX++QegbtSH3o+6mJ/gncavmig96ZI0q62EPjmHcoSak99HOfGPBOADcHuFhYms2eK1HxUURR4dmK5Jc1TmUZx7M3wVIal42rbX2l5/NPaKF74WZanxczraqcau9D39qmyVt/Lbovq+Orw0YaV5drHfq+1C/1mYqWJHvWXNSrSI+jqVNpQRHag4vuFKuhrouQWkIv3avALVZCtU3keET9QHX6hCmpQSvxJjxNr22pyKmQQNb5ysg5N15APxGQBtY1Kro9km4bewesJ0mVpWiSe8oFqtdeqINhuhpOD8DPR40ZMJgBHrhkbaQhUo0uj0AeY8wO9kVs7+O0AbMBHrN1XKsUCNhwBDDNcVJF4+jUzkN0AncGGMf2E3+XCbMAdHQBZXHfw6Wenh6nnt3ED0Q1iA84jxF1DP+dY66twL2tibWmjK3RaT5DbDtftAVYB971oPcrqvFqmgN2PEk+OUXZvOcSjvgCpHRYRBSZXNATK5sQ5XCYzmnATUffFtI23jAVcZMyPtqnqbd/qZ2wTmxO95LMVeqU4L8ZrvYnPlDHWHs7/Wies3uuNrGEwfV9VWvi9HF7OEEtISEhISEhISEhISEhISEhISEhISEhISEhISLhQ/AcKJZwHc796zgAAAABJRU5ErkJggg=='

In [ ]:
from wordcloud import WordCloud, STOPWORDS
plt.subplots(figsize=(10,10))
f1=open("xbox.png", "wb")
f1.write(codecs.decode(xbox,'base64'))
f1.close()
img1 = imread("xbox.png")
hcmask1 = img1
wordcloud = WordCloud(stopwords=STOPWORDS,background_color='black',width=1200,height=1000,mask=hcmask1
                         ).generate(" ".join(year08['title']))

plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Above is wordcloud showing the popular titles or words in the Year 2008.

## MasterPiece Analysis

In [ ]:
masterpiece=games[games['score_phrase']=='Masterpiece']
print('The total number of masterpieces produced till date are:',masterpiece.shape[0])

### Platforms with Highest Masterpieces

In [ ]:
plt.subplots(figsize=(12,6))
sns.countplot(masterpiece['platform'],palette='Set2')
plt.xticks(rotation=90)
plt.show()

Interesting To see that **Game Boy Color** has produced highest Masterpieces . Expected to see PC as the highest as it has the highest games released under its platform.

### Genres With Highest Masterpieces

In [ ]:
plt.subplots(figsize=(12,6))
sns.countplot(masterpiece['genre'],palette='Set3')
plt.xticks(rotation=90)
plt.show()

### Masterpiece Releases Per Year

In [ ]:
plt.subplots(figsize=(12,6))
sns.countplot(masterpiece['release_year'])

We can see that there were no masterpieces from 2002-2005.

In [ ]:
plt.subplots(figsize=(8,8))
f1=open("wii.png", "wb")
f1.write(codecs.decode(wii,'base64'))
f1.close()
img1 = imread("wii.png")
hcmask1 = img1
wordcloud = WordCloud(stopwords=STOPWORDS,background_color='black',width=1200,height=1000,mask=hcmask1,max_words=100
                         ).generate(" ".join(masterpiece['title']))


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Looking at the wordcloud, we can infer some the names like **Grand Theft Auto**, **Pokemon**, **Metal Gear** , etc. 

**Thanks a lot for this dataset.** 

Thank You All for having a look at this notebook. Suggestions  and Feedbacks are most welcome..